In [1]:
from datetime import datetime
from haio import (
    HAIOClient,
    Gemini_IO,
    OpenAI_IO,
    Bedrock_IO,
    QuestionTemplate,
    img_to_url,
)
import json
import sys

sys.path.append("../modules")

from virtual_human_io import VirtualHuman_IO
from vec_to_img import vec_to_img

top_dir = "/Users/pictomo/Repositories/SequentialCTA_experiment"

In [2]:
# prepare data

from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

labels = [
    "Airplane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck",
]

In [3]:
# prepare haio


virtual_human_io = VirtualHuman_IO()
gemini_io = Gemini_IO()
openai_io = OpenAI_IO()
llama_io = Bedrock_IO("us.meta.llama3-2-90b-instruct-v1:0")
nova_io = Bedrock_IO("us.amazon.nova-lite-v1:0")

haio_client = HAIOClient(
    filepath=f"{top_dir}/sandbox",
    human_io=virtual_human_io,
    openai_io=openai_io,
    gemini_io=gemini_io,
    llama_io=llama_io,
    nova_io=nova_io,
    # # claude_io=claude_io,
)

question_template: QuestionTemplate = QuestionTemplate(
    title="CIFAR-10 Image Classification",
    description="Classify the image using the CIFAR-10 dataset",
    question=[
        {"tag": "img", "src": 0},
        {
            "tag": "h1",
            "value": "Choose the label that best describes the image from the options.",
        },
        {"tag": "p", "value": " ".join(labels)},
    ],
    answer={
        "type": "select",
        "options": labels,
    },
)

In [4]:
import hashlib

async def expr1(
    execution_config: dict, task_count: int, task_phase_size: int = 1
) -> dict:
    result = {"question_count": [0], "human_count": [0], "collect_count": [0]}

    asked_questions = []
    for i in range(task_count):
        print("task:",i)
        # print(hashlib.md5(x_train[i].tobytes()).hexdigest())
        img_url = img_to_url(img_data=vec_to_img(x_train[i]), mime_type="image/png")
        # print(hashlib.md5(vec_to_img(x_train[i])).hexdigest())
        data_list = [img_url]
        asked_questions.append(
            haio_client.ask(
                question_template=question_template,
                data_list=data_list,
            )
        )
        if len(asked_questions) >= task_phase_size or i == task_count - 1:
            answer_info: HAIOClient.MethodReturn = await haio_client.wait(
                asked_questions=asked_questions, execution_config=execution_config
            )
            result["question_count"].append(
                result["question_count"][-1] + len(asked_questions)
            )
            result["human_count"].append(
                result["human_count"][-1] + answer_info["add_human_assign"]
            )
            collect_count = 0
            for j in range(len(asked_questions)):
                if (
                    answer_info["answer_list"][j]
                    == labels[y_train[i - len(asked_questions) + 1 + j][0]]
                ):
                    collect_count += 1
            result["collect_count"].append(result["collect_count"][-1] + collect_count)
            asked_questions = []

    return result

In [ ]:
from ensure_dir import ensure_dir

ensure_dir(f"{top_dir}/results/expr1")

execution_config_list = [
    ({"method": "sequential_cta_1"}, "cta1"),
    ({"method": "sequential_gta_1"}, "gta1"),
    ({"method": "sequential_cta_2"}, "cta2"),
    ({"method": "sequential_gta_2"}, "gta2"),
    ({"method": "sequential_cta_3"}, "cta3"),
    ({"method": "sequential_gta_3"}, "gta3"),
]
for task_phase_size in [1]:
    for quality_requirement in [0.8, 0.85, 0.9, 0.925, 0.95, 0.975]:
        for execution_config, name in execution_config_list:
            print(name, task_phase_size, quality_requirement)
            haio_client.del_cache_use_hist()
            haio_client.reset_state()
            execution_config["quality_requirement"] = quality_requirement
            if name in ["cta2", "gta2"]:
                # execution_config["sample_size"] = {0.8: 400, 0.85: 500, 0.9: 600, 0.95: 700}[quality_requirement]
                execution_config["sample_size"] = 500
            timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
            result = await expr1(execution_config=execution_config, task_count=10000, task_phase_size=task_phase_size)
            with open(
                f"{top_dir}/results/expr1/{name}_{task_phase_size}_{quality_requirement}_{timestamp}.json", "w"
            ) as f:
                json.dump(result, f)